In [65]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [94]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[numerator(q),denominator(q)] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R.<x> = PolynomialRing(K)
        if (x**2 - n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [67]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [68]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [69]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [70]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    K = containing_field(SGA)
    return matrix(K,[flatten([hat(delta(g),partition,SGA,K,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]).transpose()

In [71]:
SGA = SymmetricGroupAlgebra(QQ,3)

In [95]:
K = containing_field(SGA); sqrt(K(3))

-sqrt3

In [73]:
G = SGA.group()

In [74]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [75]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [76]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [77]:
specht_module = SGA.specht_module(partition)

In [78]:
rho = specht_module.representation_matrix

In [79]:
group_size = SGA.group().cardinality()

In [80]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Rational Field

In [81]:
Q = unitary_change_of_basis(SGA,partition,K); Q

[-1/6*sqrt2*sqrt3 - 1/2*sqrt2  1/6*sqrt2*sqrt3 - 1/2*sqrt2]
[ 1/6*sqrt2*sqrt3 - 1/2*sqrt2 -1/6*sqrt2*sqrt3 - 1/2*sqrt2]

In [82]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/3

In [83]:
K

Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field

In [84]:
G[3]

[2, 3, 1]

In [85]:
rho(G[3])

[-1  1]
[-1  0]

In [86]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[      -1/2  1/2*sqrt3]
[-1/2*sqrt3       -1/2]

In [87]:
U_rho*U_rho.conjugate().transpose()

[1 0]
[0 1]

In [88]:
unitary_dft(SGA)

[-1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3]
[      -1/3*sqrt3                0             -1/2        1/6*sqrt3        1/6*sqrt3              1/2]
[               0       -1/3*sqrt3        1/6*sqrt3             -1/2              1/2        1/6*sqrt3]
[               0       -1/3*sqrt3        1/6*sqrt3              1/2             -1/2        1/6*sqrt3]
[      -1/3*sqrt3                0              1/2        1/6*sqrt3        1/6*sqrt3             -1/2]
[-1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3]

In [89]:
U_dft = unitary_dft(SGA); U_dft*U_dft.conjugate().transpose()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [90]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [91]:
#what are the eigenvalues?

In [96]:
U_dft.charpoly()

x^6 + (-1/3*sqrt3 - 1/2)*x^5 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^4 + ((1/3*sqrt2 + 1/3)*sqrt3 + 1/2*sqrt2)*x^3 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^2 + (-1/3*sqrt3 - 1/2)*x + 1

In [97]:
eigs = matrix(QQbar,U_dft).eigenvalues(extend=False); eigs

ValueError: need a real or complex embedding to convert a non rational element of a number field into an algebraic number

In [ ]:
eigs

In [ ]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))